In [3]:
%cd ~/repo/protein-transfer

/home/t-fli/repo/protein-transfer


In [1]:
import lmdb

In [4]:
env = lmdb.open(str("data/structure/secondary_structure/secondary_structure_train.lmdb"), max_readers=1, readonly=True, lock=False, readahead=False, meminit=False)

In [5]:
import pickle as pkl

In [6]:
with env.begin(write=False) as txn:
    num_examples = pkl.loads(txn.get(b'num_examples'))

In [7]:
num_examples

8678

In [8]:
with env.begin(write=False) as txn:
    item = pkl.loads(txn.get(str(0).encode()))

In [9]:
item.keys()

dict_keys(['asa_max', 'disorder', 'evolutionary', 'interface', 'phi', 'primary', 'psi', 'rsa', 'ss3', 'ss8', 'valid_mask', 'id', 'protein_length'])